In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pybaseball import statcast
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
#df= statcast(start_dt='2022-03-20', end_dt='2022-10-31')

In [ ]:
#df.columns

In [ ]:
#dft = df[['player_name','p_throws','pitch_type','release_speed','release_pos_x','release_pos_z','events','description','release_spin_rate','release_extension','plate_x','plate_z']]

In [ ]:
#creating a new column for the vertical and horizontal break in inches
#dft.assign(pfx_z=df['pfx_z']*12)
#dft.assign(pfx_x=df['pfx_x']*12)
#dft.insert(5, 'Vertical Break', df['pfx_z']*12)
#dft.insert(6, 'Horizontal Break', df['pfx_x']*12)

In [ ]:
#dft.rename(columns={'player_name':'Player Name','p_throws':'Throws','pitch_type':'Pitch Type','release_speed':'Velocity','release_pos_x':'Side Release','release_pos_z':'Release Height','events':'Events','description':'Description','release_spin_rate':'Spin Rate','release_extension':'Extension','pfx_x':'Horizontal Break','pfx_z':'Vertical Break','plate_x':'Side Location','plate_z':'Height Location'}, inplace=True)
#dft = dft[['Player Name','Throws','Pitch Type','Velocity','Spin Rate','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]


In [ ]:
df = pd.read_csv('2022.csv')

In [ ]:
df

In [ ]:
#multply Horizontal Break by -1 to get the correct value
df['Horizontal Break'] = df['Horizontal Break'] * -1
#muliply Side Release by -1 to get the correct value
df['Side Release'] = df['Side Release'] * -1

In [ ]:
#load other data
pf = pd.read_csv('2023 College TM.csv')

In [ ]:
pf.head()

In [ ]:
#Create a new dataframe with only the columns we want for Analysis
pft = pf[['Pitcher','PitcherThrows','AutoPitchType','RelSpeed','SpinRate','Tilt','RelHeight','RelSide','Extension','InducedVertBreak', 'HorzBreak']]


In [ ]:
#deletes all rows with NaN values in the dataframe
pft = pft.dropna()

In [ ]:
#rename columns
pft.rename(columns={'Pitcher':'Player Name','PitcherThrows':'Throws','AutoPitchType':'Pitch Type','RelSpeed':'Velocity','SpinRate':'Spin Rate','Tilt':'Tilt','InducedVertBreak':'Vertical Break','HorzBreak':'Horizontal Break','RelHeight':'Release Height','RelSide':'Side Release','Extension':'Extension'}, inplace=True)
pft.head()

In [ ]:
#delete rows with NaN values
pft = pft.dropna()
pft.head()


In [ ]:
df.head()

In [ ]:
#rename throws values to match the other dataframe
df['Throws'] = df['Throws'].replace(['R'],'Right')
df['Throws'] = df['Throws'].replace(['L'],'Left')
df.head()

In [ ]:
#delete Unnamed:0 column
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df

In [ ]:
#in df, multiply horizontal break by -1 to match the other dataframe


In [ ]:
pft

In [ ]:
#concatenate the two dataframes
df = pd.concat([df,pft], ignore_index=True)

In [ ]:
df

In [ ]:
#delete Tilt
df = df.drop(['Tilt'], axis=1)

In [ ]:
#group by player name and pitch type without heirarchy include Throws column
df_avg_pitch = df.groupby(['Player Name','Pitch Type','Throws'], as_index=False).mean()

In [ ]:
df_avg_pitch.head()

In [ ]:
##################

In [ ]:
#rename pitch types
df['Pitch Type'] = df['Pitch Type'].replace(['FF'],'Four-Seam')
df['Pitch Type'] = df['Pitch Type'].replace(['SL'],'Slider')
df['Pitch Type'] = df['Pitch Type'].replace(['CH'],'Changeup')
df['Pitch Type'] = df['Pitch Type'].replace(['CU'],'Curveball')
df['Pitch Type'] = df['Pitch Type'].replace(['FT'],'Two-Seam Fastball')
df['Pitch Type'] = df['Pitch Type'].replace(['FC'],'Cutter')
df['Pitch Type'] = df['Pitch Type'].replace(['FS'],'Splitter')
df['Pitch Type'] = df['Pitch Type'].replace(['SI'],'Sinker')
df['Pitch Type'] = df['Pitch Type'].replace(['KC'],'Knuckle Curve')
df.head()


In [ ]:
#csv to desktop
df.to_csv('2022_combines.csv')

In [ ]:
df_avg = pd.read_csv('2022_combines.csv')

In [ ]:
#unique pitch types
df_avg['Pitch Type'].unique()

In [ ]:
#remove pitch types that are acronyms
df_avg = df_avg[df_avg['Pitch Type'] != 'ST']
df_avg = df_avg[df_avg['Pitch Type'] != 'PO']
df_avg = df_avg[df_avg['Pitch Type'] != 'SV']   
df_avg = df_avg[df_avg['Pitch Type'] != 'CS']
df_avg = df_avg[df_avg['Pitch Type'] != 'FA']
df_avg = df_avg[df_avg['Pitch Type'] != 'EP']
df_avg = df_avg[df_avg['Pitch Type'] != 'nan']
df_avg = df_avg[df_avg['Pitch Type'] != 'KN']
df_avg = df_avg[df_avg['Pitch Type'] != 'SC']
df_avg = df_avg[df_avg['Pitch Type'] != 'Other']

In [ ]:
#remove pitch types that are acronyms in df
df = df[df['Pitch Type'] != 'ST']
df = df[df['Pitch Type'] != 'PO']
df = df[df['Pitch Type'] != 'SV']
df = df[df['Pitch Type'] != 'CS']
df = df[df['Pitch Type'] != 'FA']
df = df[df['Pitch Type'] != 'EP']
df = df[df['Pitch Type'] != 'nan']
df = df[df['Pitch Type'] != 'KN']
df = df[df['Pitch Type'] != 'SC'] 
df = df[df['Pitch Type'] != 'Other']

In [ ]:
df['Pitch Type'].unique()

In [ ]:
#remove nan values
df = df.dropna()


In [ ]:
df['Pitch Type'].unique()

In [ ]:
#seperate df_avg into different dataframes for each pitch type four seam sinket together
df_avg_fastball = df_avg[df_avg['Pitch Type'].isin(['Four-Seam','Two-Seam Fastball','Sinker'])]
df_avg_slider = df_avg[df_avg['Pitch Type'].isin(['Slider'])]
df_avg_changeup = df_avg[df_avg['Pitch Type'].isin(['Changeup'])]
df_avg_curveball = df_avg[df_avg['Pitch Type'].isin(['Curveball'])]

In [ ]:
#sepereate df into different dataframes for each pitch type four seam sinker together
df_fastball = df[df['Pitch Type'].isin(['Four-Seam','Two-Seam Fastball','Sinker'])]
df_slider = df[df['Pitch Type'].isin(['Slider'])]
df_changeup = df[df['Pitch Type'].isin(['Changeup'])]
df_curveball = df[df['Pitch Type'].isin(['Curveball'])]


In [ ]:
#check the value type of df_avg_fastball
df.dtypes

In [ ]:
#export to csv
df_avg_fastball.to_csv('2022_fastball.csv')
df_avg_slider.to_csv('2022_slider.csv')
df_avg_changeup.to_csv('2022_changeup.csv')
df_avg_curveball.to_csv('2022_curveball.csv')


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans


In [ ]:
data_fb = pd.read_csv('2022_fastball.csv')

In [ ]:

#remove Cluster
#df_fastball = df_fastball.drop(['Cluster'], axis=1)

In [ ]:
#delete Unnamed: 0 column and Unnamed: 0.1 columm
data_fb = data_fb.drop(['Unnamed: 0'], axis=1)
data_fb = data_fb.drop(['Unnamed: 0.1'], axis=1)

In [ ]:

# Define which columns need to be normalized and which need to be one-hot encoded
num_cols = ['Velocity', 'Spin Rate', 'Vertical Break', 'Horizontal Break', 'Release Height', 'Side Release', 'Extension']
cat_cols = ['Player Name','Throws', 'Pitch Type']

In [ ]:
#remove nan values
df = df.dropna()

In [ ]:
#unique
df['Throws'].unique()

In [ ]:
#sepereate by Throws == Right and Throws == Left
df_right = df[df['Throws'].isin(['Right'])]
df_left = df[df['Throws'].isin(['Left'])]

In [ ]:
#unique pitchj types
df_right['Pitch Type'].unique()

In [ ]:
#sepereate by pitch type
df_right_fb = df_right[df_right['Pitch Type'].isin(['Four-Seam','Two-Seam Fastball','Sinker'])]
df_right_slider = df_right[df_right['Pitch Type'].isin(['Slider'])]
df_right_changeup = df_right[df_right['Pitch Type'].isin(['Changeup'])]
df_right_curveball = df_right[df_right['Pitch Type'].isin(['Curveball','Knuckle Curve'])]

In [ ]:
#sepereate by pitch type
df_left_fb = df_left[df_left['Pitch Type'].isin(['Four-Seam','Two-Seam Fastball','Sinker'])]
df_left_slider = df_left[df_left['Pitch Type'].isin(['Slider'])]
df_left_changeup = df_left[df_left['Pitch Type'].isin(['Changeup'])]
df_left_curveball = df_left[df_left['Pitch Type'].isin(['Curveball','Knuckle Curve'])]

In [ ]:
#create a new column nexted to velocity called Velo_percentile
df_right_fb['Velo_percentile'] = df_right_fb['Velocity'].rank(pct=True)
df_right_slider['Velo_percentile'] = df_right_slider['Velocity'].rank(pct=True)
df_right_changeup['Velo_percentile'] = df_right_changeup['Velocity'].rank(pct=True)
df_right_curveball['Velo_percentile'] = df_right_curveball['Velocity'].rank(pct=True)

In [ ]:
df_left_fb['Velo_percentile'] = df_left_fb['Velocity'].rank(pct=True)
df_left_slider['Velo_percentile'] = df_left_slider['Velocity'].rank(pct=True)
df_left_changeup['Velo_percentile'] = df_left_changeup['Velocity'].rank(pct=True)
df_left_curveball['Velo_percentile'] = df_left_curveball['Velocity'].rank(pct=True)

In [ ]:
#give percentile to extension
df_right_fb['Extension_percentile'] = df_right_fb['Extension'].rank(pct=True)
df_right_slider['Extension_percentile'] = df_right_slider['Extension'].rank(pct=True)
df_right_changeup['Extension_percentile'] = df_right_changeup['Extension'].rank(pct=True)
df_right_curveball['Extension_percentile'] = df_right_curveball['Extension'].rank(pct=True)

In [ ]:
df_left_fb['Extension_percentile'] = df_left_fb['Extension'].rank(pct=True)
df_left_slider['Extension_percentile'] = df_left_slider['Extension'].rank(pct=True)
df_left_changeup['Extension_percentile'] = df_left_changeup['Extension'].rank(pct=True)
df_left_curveball['Extension_percentile'] = df_left_curveball['Extension'].rank(pct=True)


In [ ]:
#take the mean and group by player name no multi index / hierarchy
df_right_fb = df_right_fb.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_right_slider = df_right_slider.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_right_changeup = df_right_changeup.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_right_curveball = df_right_curveball.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()

In [ ]:
df_left_fb = df_left_fb.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_left_slider = df_left_slider.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_left_changeup = df_left_changeup.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_left_curveball = df_left_curveball.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()

In [ ]:
#create a new column next to player name called player name ID and set it equal to index
df_right_fb['Player Name ID'] = df_right_fb.index
df_right_slider['Player Name ID'] = df_right_slider.index
df_right_changeup['Player Name ID'] = df_right_changeup.index
df_right_curveball['Player Name ID'] = df_right_curveball.index
df_left_fb['Player Name ID'] = df_left_fb.index
df_left_slider['Player Name ID'] = df_left_slider.index
df_left_changeup['Player Name ID'] = df_left_changeup.index
df_left_curveball['Player Name ID'] = df_left_curveball.index

In [ ]:
df_right_fb_velo = df_right_fb[['Player Name','Player Name ID','Velo_percentile']]
df_right_slider_velo = df_right_slider[['Player Name','Player Name ID','Velo_percentile']]
df_right_changeup_velo = df_right_changeup[['Player Name','Player Name ID','Velo_percentile']]
df_right_curveball_velo = df_right_curveball[['Player Name','Player Name ID','Velo_percentile']]

In [ ]:
df_left_fb_velo = df_left_fb[['Player Name','Player Name ID','Velo_percentile']]
df_left_slider_velo = df_left_slider[['Player Name','Player Name ID','Velo_percentile']]
df_left_changeup_velo = df_left_changeup[['Player Name','Player Name ID','Velo_percentile']]
df_left_curveball_velo = df_left_curveball[['Player Name','Player Name ID','Velo_percentile']]

In [ ]:
df_right_fb_movement = df_right_fb[['Vertical Break','Horizontal Break','Player Name ID']]
df_right_slider_movement = df_right_slider[['Vertical Break','Horizontal Break','Player Name ID']]
df_right_changeup_movement = df_right_changeup[['Vertical Break','Horizontal Break','Player Name ID']] 
df_right_curveball_movement = df_right_curveball[['Vertical Break','Horizontal Break','Player Name ID']]


In [ ]:
df_left_fb_movement = df_left_fb[['Vertical Break','Horizontal Break','Player Name ID']]
df_left_slider_movement = df_left_slider[['Vertical Break','Horizontal Break','Player Name ID']]
df_left_changeup_movement = df_left_changeup[['Vertical Break','Horizontal Break','Player Name ID']]
df_left_curveball_movement = df_left_curveball[['Vertical Break','Horizontal Break','Player Name ID']]

In [ ]:
df_right_fb_release = df_right_fb[['Release Height','Side Release','Player Name ID']]
df_right_slider_release = df_right_slider[['Release Height','Side Release','Player Name ID']]
df_right_changeup_release = df_right_changeup[['Release Height','Side Release','Player Name ID']]
df_right_curveball_release = df_right_curveball[['Release Height','Side Release','Player Name ID']] 


In [ ]:
df_left_fb_release = df_left_fb[['Release Height','Side Release','Player Name ID']]
df_left_slider_release = df_left_slider[['Release Height','Side Release','Player Name ID']]
df_left_changeup_release = df_left_changeup[['Release Height','Side Release','Player Name ID']]
df_left_curveball_release = df_left_curveball[['Release Height','Side Release','Player Name ID']]

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_fb_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_fb_movement['Cluster'] = movement_labels


In [ ]:
#cluster summary
df_right_fb_movement.groupby('Cluster').mean()

In [ ]:
#rename clusters based on pitch description
df_right_fb_movement['Cluster'] = df_right_fb_movement['Cluster'].replace({0:'Slight-Ride',1:'Slight Sink ',2:'Cut-Ride',3:'Sink',4:'Ride'})

In [ ]:
#value counts by cluster
df_right_fb_movement['Cluster'].value_counts()

In [ ]:
#Change the name of cluster column to Pitch Movement
df_right_fb_movement = df_right_fb_movement.rename(columns={'Cluster':'Pitch Movement'})

In [ ]:
#replace Player Name ID with Player Name from df_right_fb
df_right_fb_movement = df_right_fb_movement.replace({'Player Name ID':df_right_fb['Player Name']})

In [ ]:
#export to csv
df_right_fb_movement.to_csv('2022_right_fb_movement.csv')

In [ ]:
#import csv
df_right_fb_movement = pd.read_csv('2022_right_fb_movement.csv')

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data2 = scaler.fit_transform(df_right_fb_release)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data2)


# Add the labels to your dataframe
df_right_fb_release['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_right_fb_release.groupby('Cluster').mean()

In [ ]:
#count of clusters
df_right_fb_release['Cluster'].value_counts()

In [ ]:
#rename clusters based on Release description
df_right_fb_release['Cluster'] = df_right_fb_release['Cluster'].replace({0:'3/4',1:'3/4',2:'OverTheTop',3:'Low 3/4 | Sidearm',4:'OverTheTop'})

In [ ]:
#Change the name of cluster column to Pitch Release
df_right_fb_release = df_right_fb_release.rename(columns={'Cluster':'Pitch Release'})

In [ ]:
#replace Player Name ID with Player Name from df_right_fb
df_right_fb_release = df_right_fb_release.replace({'Player Name ID':df_right_fb['Player Name']})

In [ ]:
#export to csv
df_right_fb_release.to_csv('2022_right_fb_release.csv')

In [ ]:
#import csv
df_right_fb_release = pd.read_csv('2022_right_fb_release.csv')

In [ ]:
#combine movement and release dataframes for right handed fastballs based on Player Name ID
df_right_fb_movement_release = pd.merge(df_right_fb_movement, df_right_fb_release, on='Player Name ID', how='outer')

In [ ]:
#df_right_fb_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_right_fb_velo['Cluster'] = pd.cut(df_right_fb_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])



In [ ]:
#rename clusters based on velo percentile
df_right_fb_velo['Cluster'] = df_right_fb_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

In [ ]:
#rename cluster column to Pitch Velocity description
df_right_fb_velo = df_right_fb_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [ ]:
#replace Player Name ID with Player Name from df_right_fb
df_right_fb_velo = df_right_fb_velo.replace({'Player Name ID':df_right_fb['Player Name']})

In [ ]:
#combine movement, release, and velo dataframes for right handed fastballs based on Player Name ID
df_right_fb_movement_release_velo = pd.merge(df_right_fb_movement_release, df_right_fb_velo, on='Player Name ID', how='outer')

In [ ]:
#df_right_fb_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_right_fb_movement_release_velo = df_right_fb_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]


In [ ]:
# rename Player Name ID to Player Name in df_right_fb_movement_release_velo
df_right_fb_movement_release_velo = df_right_fb_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [ ]:
#df_right_fb2
#df_right_fb only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_right_fb2 = df_right_fb[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]

In [ ]:
#combine df_right_fb2 and df_right_fb_movement_release_velo if name appears in both dataframes
df_right_fb_final = pd.merge(df_right_fb2, df_right_fb_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_right_fb_final
df_right_fb_final = df_right_fb_final.drop_duplicates(subset=['Player Name'])

In [ ]:
#sort high to low by velocity
df_right_fb_final = df_right_fb_final.sort_values(by=['Velocity'], ascending=False)

In [ ]:
######BEGIN SLIDER##############################################################################################

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_slider_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_slider_movement['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_right_slider_movement.groupby('Cluster').mean()

In [ ]:
#value counts by cluster
df_right_slider_movement['Cluster'].value_counts()

In [ ]:
#rename clusters based on pitch description
df_right_slider_movement['Cluster'] = df_right_slider_movement['Cluster'].replace({0:'Gyro',1:'Sweeper',2:'Gyro',3:'Cutter'})

In [ ]:
#Change the name of cluster column to Pitch Movement
df_right_slider_movement = df_right_slider_movement.rename(columns={'Cluster':'Pitch Movement'})

In [ ]:
#replace Player Name ID with Player Name from df_right_slider
df_right_slider_movement = df_right_slider_movement.replace({'Player Name ID':df_right_slider['Player Name']})

In [ ]:
#export to csv
df_right_slider_movement.to_csv('2022_right_slider_movement.csv')

In [ ]:
#import csv
df_right_slider_movement = pd.read_csv('2022_right_slider_movement.csv')

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_slider_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_slider_release['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_right_slider_release.groupby('Cluster').mean()

In [ ]:
#count of clusters
df_right_slider_release['Cluster'].value_counts()

In [ ]:
#rename clusters based on Release description
df_right_slider_release['Cluster'] = df_right_slider_release['Cluster'].replace({0:'Low 3/4 | Sidearm',1:'3/4',2:'3/4',3:'OverTheTop'})

In [ ]:
#Change the name of cluster column to Pitch Release
df_right_slider_release = df_right_slider_release.rename(columns={'Cluster':'Pitch Release'})

In [ ]:
#replace Player Name ID with Player Name from df_right_slider
df_right_slider_release = df_right_slider_release.replace({'Player Name ID':df_right_slider['Player Name']})

In [ ]:
#export to csv
df_right_slider_release.to_csv('2022_right_slider_release.csv')

In [ ]:
#import csv
df_right_slider_release = pd.read_csv('2022_right_slider_release.csv')

In [ ]:
#combine movement and release dataframes for right handed sliders based on Player Name ID
df_right_slider_movement_release = pd.merge(df_right_slider_movement, df_right_slider_release, on='Player Name ID', how='outer')

In [ ]:
#remove Unnamed: 0 column
df_right_slider_movement_release = df_right_slider_movement_release.drop(['Unnamed: 0_x'], axis=1)
df_right_slider_movement_release = df_right_slider_movement_release.drop(['Unnamed: 0_y'], axis=1)

In [ ]:
df_right_slider_velo['Cluster'] = pd.cut(df_right_slider_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

In [ ]:
#rename clusters based on velo percentile
df_right_slider_velo['Cluster'] = df_right_slider_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

In [ ]:
#rename cluster column to Pitch Velocity description
df_right_slider_velo = df_right_slider_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [ ]:
#replace Player Name ID with Player Name from df_right_slider
df_right_slider_velo = df_right_slider_velo.replace({'Player Name ID':df_right_slider['Player Name']})

In [ ]:
#combine movement, release, and velo dataframes for right handed sliders based on Player Name ID
df_right_slider_movement_release_velo = pd.merge(df_right_slider_movement_release, df_right_slider_velo, on='Player Name ID', how='outer')

In [ ]:
#df_right_slider_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_right_slider_movement_release_velo = df_right_slider_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]

In [ ]:
# rename Player Name ID to Player Name in df_right_slider_movement_release_velo
df_right_slider_movement_release_velo = df_right_slider_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [ ]:
#df_right_slider2
#df_right_slider only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_right_slider2 = df_right_slider[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]


In [ ]:
#combine df_right_fb2 and df_right_fb_movement_release_velo if name appears in both dataframes
df_right_slider_final = pd.merge(df_right_slider2, df_right_slider_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_right_fb_final
df_right_slider_final = df_right_slider_final.drop_duplicates(subset=['Player Name'])

In [ ]:
#BEGIN CHANGEUP######################################################################################################

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_changeup_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_changeup_movement['Cluster'] = movement_labels

In [ ]:
df_right_changeup_movement

In [ ]:
#summary of clusters
df_right_changeup_movement.groupby('Cluster').mean()

In [ ]:
#value counts by cluster
df_right_changeup_movement['Cluster'].value_counts()

In [ ]:
#rename clusters based on pitch description
df_right_changeup_movement['Cluster'] = df_right_changeup_movement['Cluster'].replace({0:'FB Shape',1:'Depth w/ Fade',2:'FB Shape',3:'Depth w/ fade',4:'Cut-Change'})

In [ ]:
#change the name of cluster column to Pitch Movement
df_right_changeup_movement = df_right_changeup_movement.rename(columns={'Cluster':'Pitch Movement'})

In [ ]:
#replace Player Name ID with Player Name from df_right_changeup
df_right_changeup_movement = df_right_changeup_movement.replace({'Player Name ID':df_right_changeup['Player Name']})

In [ ]:
#export to csv
df_right_changeup_movement.to_csv('2022_right_changeup_movement.csv')


In [ ]:
#import csv
df_right_changeup_movement = pd.read_csv('2022_right_changeup_movement.csv')

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_changeup_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_changeup_release['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_right_changeup_release.groupby('Cluster').mean()

In [ ]:
#count of clusters
df_right_changeup_release['Cluster'].value_counts()

In [ ]:
#rename clusters based on Release description
df_right_changeup_release['Cluster'] = df_right_changeup_release['Cluster'].replace({0:'3/4',1:'3/4',2:'Low 3/4 | Sidearm',3:'OverTheTop'})

In [ ]:
#Change the name of cluster column to Pitch Release
df_right_changeup_release = df_right_changeup_release.rename(columns={'Cluster':'Pitch Release'})

In [ ]:
#replace Player Name ID with Player Name from df_right_changeup
df_right_changeup_release = df_right_changeup_release.replace({'Player Name ID':df_right_changeup['Player Name']})

In [ ]:
#export to csv
df_right_changeup_release.to_csv('2022_right_changeup_release.csv')

In [ ]:
#import csv
df_right_changeup_release = pd.read_csv('2022_right_changeup_release.csv')

In [ ]:
#combine movement and release dataframes for right handed changeups based on Player Name ID
df_right_changeup_movement_release = pd.merge(df_right_changeup_movement, df_right_changeup_release, on='Player Name ID', how='outer')

In [ ]:
df_right_changeup_velo['Cluster'] = pd.cut(df_right_changeup_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

In [ ]:
#rename clusters based on velo percentile
df_right_changeup_velo['Cluster'] = df_right_changeup_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

In [ ]:
#rename cluster column to Pitch Velocity description
df_right_changeup_velo = df_right_changeup_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [ ]:
#replace Player Name ID with Player Name from df_right_changeup
df_right_changeup_velo = df_right_changeup_velo.replace({'Player Name ID':df_right_changeup['Player Name']})

In [ ]:
#combine movement, release, and velo dataframes for right handed changeups based on Player Name ID
df_right_changeup_movement_release_velo = pd.merge(df_right_changeup_movement_release, df_right_changeup_velo, on='Player Name ID', how='outer')

In [ ]:
#df_right_changeup_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_right_changeup_movement_release_velo = df_right_changeup_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]

In [ ]:
# rename Player Name ID to Player Name in df_right_changeup_movement_release_velo
df_right_changeup_movement_release_velo = df_right_changeup_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [ ]:
#df_right_changeup2
#df_right_changeup only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_right_changeup2 = df_right_changeup[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]

In [ ]:
#combine df_right_changeup2 and df_right_changeup_movement_release_velo if name appears in both dataframes
df_right_changeup_final = pd.merge(df_right_changeup2, df_right_changeup_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_right_changeup_final
df_right_changeup_final = df_right_changeup_final.drop_duplicates(subset=['Player Name'])

In [ ]:
#sort high to low by velocity
df_right_changeup_final = df_right_changeup_final.sort_values(by=['Velocity'], ascending=False)


In [ ]:
#BEGIN CURVEBALL######################################################################################################

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_curveball_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=3)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_curveball_movement['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_right_curveball_movement.groupby('Cluster').mean()

In [ ]:
#value counts by cluster
df_right_curveball_movement['Cluster'].value_counts()

In [ ]:
#rename clusters based on pitch description
df_right_curveball_movement['Cluster'] = df_right_curveball_movement['Cluster'].replace({0:'Small Curve',1:'True Curve',2:'Slider Type'})

In [ ]:
#change the name of cluster column to Pitch Movement
df_right_curveball_movement = df_right_curveball_movement.rename(columns={'Cluster':'Pitch Movement'})

In [ ]:
#replace Player Name ID with Player Name from df_right_curveball
df_right_curveball_movement = df_right_curveball_movement.replace({'Player Name ID':df_right_curveball['Player Name']})

In [ ]:
#export to csv
df_right_curveball_movement.to_csv('2022_right_curveball_movement.csv')

In [ ]:
#import csv
df_right_curveball_movement = pd.read_csv('2022_right_curveball_movement.csv')

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_curveball_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_curveball_release['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_right_curveball_release.groupby('Cluster').mean()

In [ ]:
#value counts by cluster
df_right_curveball_release['Cluster'].value_counts()

In [ ]:
#rename clusters based on Release description
df_right_curveball_release['Cluster'] = df_right_curveball_release['Cluster'].replace({0:'3/4',1:'3/4',2:'Low 3/4 | Sidearm',3:'OverTheTop'})

In [ ]:
#Change the name of cluster column to Pitch Release
df_right_curveball_release = df_right_curveball_release.rename(columns={'Cluster':'Pitch Release'})

In [ ]:
#replace Player Name ID with Player Name from df_right_changeup
df_right_curveball_release = df_right_curveball_release.replace({'Player Name ID':df_right_curveball['Player Name']})

In [ ]:
#export to csv
df_right_curveball_release.to_csv('2022_right_curveball_release.csv')

In [ ]:
#import csv
df_right_curveball_release = pd.read_csv('2022_right_curveball_release.csv')

In [ ]:
#combine movement and release dataframes for right handed changeups based on Player Name ID
df_right_curveball_movement_release = pd.merge(df_right_curveball_movement, df_right_curveball_release, on='Player Name ID', how='outer')

In [ ]:
df_right_curveball_velo['Cluster'] = pd.cut(df_right_curveball_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

In [ ]:
#rename clusters based on velo percentile
df_right_curveball_velo['Cluster'] = df_right_curveball_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

In [ ]:
#rename cluster column to Pitch Velocity description
df_right_curveball_velo = df_right_curveball_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [ ]:
#replace Player Name ID with Player Name from df_right_curveball
df_right_curveball_velo = df_right_curveball_velo.replace({'Player Name ID':df_right_curveball['Player Name']})

In [ ]:
#combine movement, release, and velo dataframes for right handed curveballs based on Player Name ID
df_right_curveball_movement_release_velo = pd.merge(df_right_curveball_movement_release, df_right_curveball_velo, on='Player Name ID', how='outer')

In [ ]:
#df_right_curveball_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_right_curveball_movement_release_velo = df_right_curveball_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]

In [ ]:
#player Name ID to Player Name in df_right_changeup_movement_release_velo
df_right_curveball_movement_release_velo = df_right_curveball_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [ ]:
#df_right_curveball2
#df_right_curveball only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_right_curveball2 = df_right_curveball[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]

In [ ]:
#combine df_right_curveball2 and df_right_curveball_movement_release_velo if name appears in both dataframes
df_right_curveball_final = pd.merge(df_right_curveball2, df_right_curveball_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_right_changeup_final
df_right_curveball_final = df_right_curveball_final.drop_duplicates(subset=['Player Name'])

In [ ]:
# create a new column in df_right_fb_final called Pitch Type and set it equal to FB
df_right_fb_final['Pitch Type'] = 'FB'
df_right_slider_final['Pitch Type'] = 'SL'
df_right_changeup_final['Pitch Type'] = 'CH'
df_right_curveball_final['Pitch Type'] = 'CB'

In [ ]:
#combine all 4 dataframes into one
df_right_final = pd.concat([df_right_fb_final, df_right_slider_final, df_right_changeup_final, df_right_curveball_final])

In [ ]:
#sort Player Name alphabetically
df_right_final = df_right_final.sort_values(by=['Player Name'], ascending=True)

In [ ]:
#new dataframe with description
df_right_final_description = df_right_final[['Player Name','Throws','Pitch Type','Pitch Velocity','Pitch Movement','Pitch Release']]

In [ ]:
#combine columns Pitch Velocity','Pitch Movement','Pitch Release' into one column called Pitch Description
df_right_final_description['Pitch Description'] = df_right_final_description['Pitch Velocity'].astype(str) + ' ' + df_right_final_description['Pitch Movement'].astype(str) + ' ' + df_right_final_description['Pitch Release'].astype(str)

In [ ]:
#only include Player Name, Throws, Pitch Type, Pitch Description
df_right_final_description = df_right_final_description[['Player Name','Throws','Pitch Type','Pitch Description']]

In [ ]:
#locate Jacob deGrom
df_right_final_description.loc[df_right_final_description['Player Name'] == 'Skenes, Paul']

In [ ]:
###BEGIN LEFTY FB###################################################################################

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_fb_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_fb_movement['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_left_fb_movement.groupby('Cluster').mean()

In [ ]:
#value counts by cluster
df_left_fb_movement['Cluster'].value_counts()

In [ ]:
#rename clusters based on pitch description
df_left_fb_movement['Cluster'] = df_left_fb_movement['Cluster'].replace({0:'Sink',1:'Cut-Ride ',2:'Slight Sink',3:'Ride',4:'Slight Ride'})

In [ ]:
#Change the name of cluster column to Pitch Movement
df_left_fb_movement = df_left_fb_movement.rename(columns={'Cluster':'Pitch Movement'})

In [ ]:
#replace Player Name ID with Player Name from df_left_fb
df_left_fb_movement = df_left_fb_movement.replace({'Player Name ID':df_left_fb['Player Name']})

In [ ]:
#export to csv
df_left_fb_movement.to_csv('2022_left_fb_movement.csv')

In [ ]:
#import csv
df_left_fb_movement = pd.read_csv('2022_left_fb_movement.csv')

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data2 = scaler.fit_transform(df_left_fb_release)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data2)


# Add the labels to your dataframe
df_left_fb_release['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_left_fb_release.groupby('Cluster').mean()

In [ ]:
#count of clusters
df_left_fb_release['Cluster'].value_counts()

In [ ]:
#rename clusters based on Release description
df_left_fb_release['Cluster'] = df_left_fb_release['Cluster'].replace({0:'3/4',1:'OverTheTop',2:'Low 3/4 | Sidearm',3:'3/4',4:'OverTheTop'})

In [ ]:
#Change the name of cluster column to Pitch Release
df_left_fb_release = df_left_fb_release.rename(columns={'Cluster':'Pitch Release'})

In [ ]:
#replace Player Name ID with Player Name from df_left_fb
df_left_fb_release = df_left_fb_release.replace({'Player Name ID':df_left_fb['Player Name']})

In [ ]:
#export to csv
df_left_fb_release.to_csv('2022_left_fb_release.csv')

In [ ]:
#import csv
df_left_fb_release = pd.read_csv('2022_left_fb_release.csv')

In [ ]:
#combine movement and release dataframes for left handed fastballs based on Player Name ID
df_left_fb_movement_release = pd.merge(df_left_fb_movement, df_left_fb_release, on='Player Name ID', how='outer')


In [ ]:
#df_left_fb_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_left_fb_velo['Cluster'] = pd.cut(df_left_fb_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

In [ ]:
#rename clusters based on velo percentile
df_left_fb_velo['Cluster'] = df_left_fb_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

In [ ]:
#rename cluster column to Pitch Velocity description
df_left_fb_velo = df_left_fb_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [ ]:
#replace Player Name ID with Player Name from df_left_fb
df_left_fb_velo = df_left_fb_velo.replace({'Player Name ID':df_left_fb['Player Name']})

In [ ]:
#combine movement, release, and velo dataframes for left handed fastballs based on Player Name ID
df_left_fb_movement_release_velo = pd.merge(df_left_fb_movement_release, df_left_fb_velo, on='Player Name ID', how='outer')

In [ ]:
#df_left_fb_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_left_fb_movement_release_velo = df_left_fb_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]

In [ ]:
# rename Player Name ID to Player Name in df_left_fb_movement_release_velo
df_left_fb_movement_release_velo = df_left_fb_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [ ]:
#df_left_fb2
#df_left_fb only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_left_fb2 = df_left_fb[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]

In [ ]:
#combine df_left_fb2 and df_left_fb_movement_release_velo if name appears in both dataframes
df_left_fb_final = pd.merge(df_left_fb2, df_left_fb_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_left_fb_final
df_left_fb_final = df_left_fb_final.drop_duplicates(subset=['Player Name'])


In [ ]:
#sort high to low by velocity
df_left_fb_final = df_left_fb_final.sort_values(by=['Velocity'], ascending=False)

In [ ]:
######BEGIN SLIDER##############################################################################################

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_slider_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_slider_movement['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_left_slider_movement.groupby('Cluster').mean()

In [ ]:
#value counts by cluster
df_left_slider_movement['Cluster'].value_counts()

In [ ]:
#rename clusters based on pitch description
df_left_slider_movement['Cluster'] = df_left_slider_movement['Cluster'].replace({0:'Gyro',1:'Cutter',2:'Gyro',3:'Sweeper'})

In [ ]:
#change the name of cluster column to Pitch Movement
df_left_slider_movement = df_left_slider_movement.rename(columns={'Cluster':'Pitch Movement'})

In [ ]:
#replace Player Name ID with Player Name from df_left_slider
df_left_slider_movement = df_left_slider_movement.replace({'Player Name ID':df_left_slider['Player Name']})

In [ ]:
#export to csv
df_left_slider_movement.to_csv('2022_left_slider_movement.csv')

In [ ]:
#import csv
df_left_slider_movement = pd.read_csv('2022_left_slider_movement.csv')

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_slider_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_slider_release['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_left_slider_release.groupby('Cluster').mean()

In [ ]:
#count of clusters
df_left_slider_release['Cluster'].value_counts()

In [ ]:
#rename clusters based on Release description
df_left_slider_release['Cluster'] = df_left_slider_release['Cluster'].replace({0:'3/4',1:'Low 3/4 | Sidearm',2:'OverTheTop',3:'3/4'})

In [ ]:
#Change the name of cluster column to Pitch Release
df_left_slider_release = df_left_slider_release.rename(columns={'Cluster':'Pitch Release'})

In [ ]:
#replace Player Name ID with Player Name from df_left_slider
df_left_slider_release = df_left_slider_release.replace({'Player Name ID':df_left_slider['Player Name']})

In [ ]:
#export to csv
df_left_slider_release.to_csv('2022_left_slider_release.csv')


In [ ]:
#import csv
df_left_slider_release = pd.read_csv('2022_left_slider_release.csv')

In [ ]:
#combine movement and release dataframes for left handed sliders based on Player Name ID
df_left_slider_movement_release = pd.merge(df_left_slider_movement, df_left_slider_release, on='Player Name ID', how='outer')

In [ ]:
#df_left_slider_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_left_slider_velo['Cluster'] = pd.cut(df_left_slider_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

In [ ]:
#rename clusters based on velo percentile
df_left_slider_velo['Cluster'] = df_left_slider_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

In [ ]:
#rename cluster column to Pitch Velocity description
df_left_slider_velo = df_left_slider_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [ ]:
#replace Player Name ID with Player Name from df_left_slider
df_left_slider_velo = df_left_slider_velo.replace({'Player Name ID':df_left_slider['Player Name']})

In [ ]:
#combine movement, release, and velo dataframes for left handed sliders based on Player Name ID
df_left_slider_movement_release_velo = pd.merge(df_left_slider_movement_release, df_left_slider_velo, on='Player Name ID', how='outer')

In [ ]:
#df_left_slider_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_left_slider_movement_release_velo = df_left_slider_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]
# rename Player Name ID to Player Name in df_left_slider_movement_release_velo
df_left_slider_movement_release_velo = df_left_slider_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})


In [ ]:
#df_left_slider2
#df_left_slider only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_left_slider2 = df_left_slider[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]
#combine df_left_slider2 and df_left_slider_movement_release_velo if name appears in both dataframes
df_left_slider_final = pd.merge(df_left_slider2, df_left_slider_movement_release_velo, on='Player Name', how='inner')

In [ ]:
#delete duplicate rows if name appears in more than once in df_left_slider_final
df_left_slider_final = df_left_slider_final.drop_duplicates(subset=['Player Name'])

In [ ]:
#sort high to low by velocity
df_left_slider_final = df_left_slider_final.sort_values(by=['Velocity'], ascending=False)

In [ ]:
######BEGIN CHANGEUP##############################################################################################

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_changeup_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_changeup_movement['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_left_changeup_movement.groupby('Cluster').mean()

In [ ]:
#value counts by cluster
df_left_changeup_movement['Cluster'].value_counts()

In [ ]:
#rename clusters based on pitch description
df_left_changeup_movement['Cluster'] = df_left_changeup_movement['Cluster'].replace({0:'Depth w/ Fade',1:'Drop',2:'FB Shape',3:'Depth w/ fade'})

In [ ]:
#change the name of cluster column to Pitch Movement
df_left_changeup_movement = df_left_changeup_movement.rename(columns={'Cluster':'Pitch Movement'})

In [ ]:
#replace Player Name ID with Player Name from df_left_changeup
df_left_changeup_movement = df_left_changeup_movement.replace({'Player Name ID':df_left_changeup['Player Name']})

In [ ]:
#show all with drop
df_left_changeup_movement.loc[df_left_changeup_movement['Pitch Movement'] == 'Drop']
# delete rows with drop
df_left_changeup_movement = df_left_changeup_movement[df_left_changeup_movement['Pitch Movement'] != 'Drop']


In [ ]:
#export to csv
df_left_changeup_movement.to_csv('2022_left_changeup_movement.csv')


In [ ]:
#import csv
df_left_changeup_movement = pd.read_csv('2022_left_changeup_movement.csv')

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_changeup_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_changeup_release['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_left_changeup_release.groupby('Cluster').mean()

In [ ]:
# count of clusters
df_left_changeup_release['Cluster'].value_counts()

In [ ]:
#rename clusters based on Release description
df_left_changeup_release['Cluster'] = df_left_changeup_release['Cluster'].replace({0:'3/4',1:'3/4',2:'Low 3/4 | Sidearm',3:'OverTheTop'})

In [ ]:
#Change the name of cluster column to Pitch Release
df_left_changeup_release = df_left_changeup_release.rename(columns={'Cluster':'Pitch Release'})

In [ ]:
#replace Player Name ID with Player Name from df_left_changeup
df_left_changeup_release = df_left_changeup_release.replace({'Player Name ID':df_left_changeup['Player Name']})

In [ ]:
#export to csv
df_left_changeup_release.to_csv('2022_left_changeup_release.csv')

In [ ]:
#import csv
df_left_changeup_release = pd.read_csv('2022_left_changeup_release.csv')

In [ ]:
#combine movement and release dataframes for left handed changeups based on Player Name ID
df_left_changeup_movement_release = pd.merge(df_left_changeup_movement, df_left_changeup_release, on='Player Name ID', how='outer')

In [ ]:
#df_left_changeup_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_left_changeup_velo['Cluster'] = pd.cut(df_left_changeup_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

In [ ]:
#rename clusters based on velo percentile
df_left_changeup_velo['Cluster'] = df_left_changeup_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

In [ ]:
#rename cluster column to Pitch Velocity description
df_left_changeup_velo = df_left_changeup_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [ ]:
#replace Player Name ID with Player Name from df_left_changeup
df_left_changeup_velo = df_left_changeup_velo.replace({'Player Name ID':df_left_changeup['Player Name']})

In [ ]:
#combine movement, release, and velo dataframes for left handed changeups based on Player Name ID
df_left_changeup_movement_release_velo = pd.merge(df_left_changeup_movement_release, df_left_changeup_velo, on='Player Name ID', how='outer')


In [ ]:
#df_left_changeup_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_left_changeup_movement_release_velo = df_left_changeup_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]
# rename Player Name ID to Player Name in df_left_changeup_movement_release_velo
df_left_changeup_movement_release_velo = df_left_changeup_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [ ]:
#df_left_changeup2
#df_left_changeup only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_left_changeup2 = df_left_changeup[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]
#combine df_left_changeup2 and df_left_changeup_movement_release_velo if name appears in both dataframes
df_left_changeup_final = pd.merge(df_left_changeup2, df_left_changeup_movement_release_velo, on='Player Name', how='inner')

In [ ]:
#delete duplicate rows if name appears in more than once in df_left_changeup_final
df_left_changeup_final = df_left_changeup_final.drop_duplicates(subset=['Player Name'])

In [ ]:
#sort high to low by velocity
df_left_changeup_final = df_left_changeup_final.sort_values(by=['Velocity'], ascending=False)

In [ ]:
######BEGIN CURVEBALL##############################################################################################

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_curveball_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=6)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_curveball_movement['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_left_curveball_movement.groupby('Cluster').mean()

In [ ]:
#value counts by cluster
df_left_curveball_movement['Cluster'].value_counts()

In [ ]:
#rename clusters based on pitch description
df_left_curveball_movement['Cluster'] = df_left_curveball_movement['Cluster'].replace({0:'Slurve',1:'True Curve',2:'Small Curve',3:'True Curve',4:'Small Curve',5:'Slider Type'})

In [ ]:
#change the name of cluster column to Pitch Movement
df_left_curveball_movement = df_left_curveball_movement.rename(columns={'Cluster':'Pitch Movement'})

In [ ]:
#replace Player Name ID with Player Name from df_left_curveball
df_left_curveball_movement = df_left_curveball_movement.replace({'Player Name ID':df_left_curveball['Player Name']})

In [ ]:
#export to csv
df_left_curveball_movement.to_csv('2022_left_curveball_movement.csv')

In [ ]:
#import csv
df_left_curveball_movement = pd.read_csv('2022_left_curveball_movement.csv')

In [ ]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_curveball_release)

# Implement KMeans
kmeans = KMeans(n_clusters=6)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_curveball_release['Cluster'] = movement_labels

In [ ]:
#summary of clusters
df_left_curveball_release.groupby('Cluster').mean()

In [ ]:
#value counts by cluster
df_left_curveball_release['Cluster'].value_counts()

In [ ]:
#rename clusters based on Release description
df_left_curveball_release['Cluster'] = df_left_curveball_release['Cluster'].replace({0:'Low 3/4 | Sidearm',1:'3/4',2:'3/4',3:'Low 3/4 | Sidearm',4:'3/4',5:'OverTheTop'})

In [ ]:
#Change the name of cluster column to Pitch Release
df_left_curveball_release = df_left_curveball_release.rename(columns={'Cluster':'Pitch Release'})

In [ ]:
#replace Player Name ID with Player Name from df_left_changeup
df_left_curveball_release = df_left_curveball_release.replace({'Player Name ID':df_left_curveball['Player Name']})

In [ ]:
#export to csv
df_left_curveball_release.to_csv('2022_left_curveball_release.csv')

In [ ]:
#import csv
df_left_curveball_release = pd.read_csv('2022_left_curveball_release.csv')

In [ ]:
#combine movement and release dataframes for left handed curveballs based on Player Name ID
df_left_curveball_movement_release = pd.merge(df_left_curveball_movement, df_left_curveball_release, on='Player Name ID', how='outer')

In [ ]:
#df_left_curveball_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_left_curveball_velo['Cluster'] = pd.cut(df_left_curveball_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

In [ ]:
#rename clusters based on velo percentile
df_left_curveball_velo['Cluster'] = df_left_curveball_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

In [ ]:
#rename cluster column to Pitch Velocity description
df_left_curveball_velo = df_left_curveball_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [ ]:
#replace Player Name ID with Player Name from df_left_curveball
df_left_curveball_velo = df_left_curveball_velo.replace({'Player Name ID':df_left_curveball['Player Name']})


In [ ]:
#combine movement, release, and velo dataframes for left handed curveballs based on Player Name ID
df_left_curveball_movement_release_velo = pd.merge(df_left_curveball_movement_release, df_left_curveball_velo, on='Player Name ID', how='outer')

In [ ]:
#df_left_curveball_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_left_curveball_movement_release_velo = df_left_curveball_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]
# rename Player Name ID to Player Name in df_left_curveball_movement_release_velo
df_left_curveball_movement_release_velo = df_left_curveball_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [ ]:
#df_left_curveball2
#df_left_curveball only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_left_curveball2 = df_left_curveball[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]
#combine df_left_curveball2 and df_left_curveball_movement_release_velo if name appears in both dataframes
df_left_curveball_final = pd.merge(df_left_curveball2, df_left_curveball_movement_release_velo, on='Player Name', how='inner')

In [ ]:
#delete duplicate rows if name appears in more than once in df_left_curveball_final
df_left_curveball_final = df_left_curveball_final.drop_duplicates(subset=['Player Name'])


In [ ]:
#sort high to low by velocity
df_left_curveball_final = df_left_curveball_final.sort_values(by=['Velocity'], ascending=False)

In [ ]:
# create a new column in df_left_fb_final called Pitch Type and set it equal to FB
df_left_fb_final['Pitch Type'] = 'FB'
df_left_slider_final['Pitch Type'] = 'SL'
df_left_changeup_final['Pitch Type'] = 'CH'
df_left_curveball_final['Pitch Type'] = 'CB'

In [ ]:
#combine all 4 dataframes into one
df_left_final = pd.concat([df_left_fb_final, df_left_slider_final, df_left_changeup_final, df_left_curveball_final])

In [ ]:
#new dataframe with description
df_left_final_description = df_left_final[['Player Name','Throws','Pitch Type','Pitch Velocity','Pitch Movement','Pitch Release']]

In [ ]:
#combine columns Pitch Velocity','Pitch Movement','Pitch Release' into one column called Pitch Description
df_left_final_description['Pitch Description'] = df_left_final_description['Pitch Velocity'].astype(str) + ' ' + df_left_final_description['Pitch Movement'].astype(str) + ' ' + df_left_final_description['Pitch Release'].astype(str)

In [ ]:
#only include Player Name, Throws, Pitch Type, Pitch Description
df_left_final_description = df_left_final_description[['Player Name','Throws','Pitch Type','Pitch Description']]

In [ ]:
#locate players
df_left_final_description.loc[df_left_final_description['Player Name'] == 'Frank, Hayden']

In [ ]:
#locate players
df_right_final_description.loc[df_right_final_description['Player Name'] == 'Brecht, Brody']

In [ ]:
#combine right and left handed pitchers
df_final = pd.concat([df_right_final_description, df_left_final_description])

In [ ]:
#look up player
df_final.loc[df_final['Player Name'] == 'Brecht, Brody']